# 正则表达式 regex

正则表达式本质上是个独立的语音，短小却格外强悍。
regular expression，pattern，match

unix sed grep

> 我们可以用书写特定的规则，用来在文本中捕获与规则一致的字符串，而后对其进行操作……

In [ ]:
import re
strs = 'The quick brown fox jumps over the lazy dog.'
pttn = re.compile(r'\wo\w')
re.findall(pttn, strs)

> **规则表达式**（regular expression，通常缩写为 Reges）是强大且不可或缺的文本处理工具——它的用处就是在文本中**扫描/搜索**（scan/search）与某一**规则**（patterh）**匹配**（match，即，与规则一致）的所有实例，并且还可以按照规则**捕获**（capture）其中的部分或者全部，对它们进行**替换**（replace）。

有时，使用 regex 并不是为了 replace，而是为了检查格式。比如，
> * 可以用 regex 检查用户输入的密码是否过于简单，比如，全部由数字构成。
> * 
> * 比如可以用来验证用户输入的电话号码、证件号码是否符合特定格式等等。

另外，在自学过程中，想尽一切办法把一切术语用简单直白的**人话**重新表述，是特别有效的促进进步的行为模式。

## 视觉体验

> 眼见为实——想办法让一个陌生的概念视觉上直观，是突破大多数学习障碍的最简单粗暴直接有效的方式。

## 准备工作

**regex-target-text-sample.txt**

In [ ]:
pwd

In [ ]:
import re
with open('regex-target-text-sample.txt') as f:
    strs = f.read()
pttn = r'beg[iau]ns?'
re.findall(pttn, strs)

## 优先级

编程语言无非是用来运算的。

所谓的运算，就有操作符（operators）和操作元（operators）——而操作符肯定是有优先级的。不然的话，那么多操作元和操作符放在一起，究竟先操作哪个呢？

regex 也一样，它本身就是个迷你语言（mini language）。在 regex 中，操作符肯定也有优先级。它的操作元有个专门的名称——**原子**（atom）。

先看看它的操作符优先级：

|排列|原子与操作符优先级 |（从高到低）|
|-|-|-|
|1|转移符号 escaping symbol|`\`|
|2|分组、捕获 grouping or capturing|`(...)` `(?:...)` `(?=...)` `(?!...)` `(?<=...)` `(?<!...)`|
|3|数量 quantifiers| `a* a+ a? a{n, m}` |
|4|序列与定位 sequence and anchor|`abc` `^` `$` `\b` `\B`|
|5|或 alternation|`a\|b\|c`|
|6|原子 atoms| `a` `[^abc]` `\t` `\r` `\n` `\d` `\D` `\s` `\S` `\w` `\W` `.`|

如果在之前，没有自学过、理解过 python（或者任何其他编程语言）表达式中的操作符优先级，那么一上来就看上面的表格不仅对你没有帮助，只能让你迷惑。

——**这就是理解能力逐步积累逐步加强的过程**。

## 原子

在 regex 的 pattern 中，操作元，即，被运算的**值**，被称为**原子**（atom）。

### 本义字符

最基本的原子，就是本义字符，它们都是单个字符。
本义字符包括从 `a` 到 `z`， `A` 到 `Z`， `0` 到 `9`，还有 `_`——它们所代表的就是它们的字面值。

> 即，相当于，`string.ascii_letters` 和 `string.digits`，以及 `_`。

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import string
string.ascii_letters
string.digits

一下字符在 regex 中都有特殊的含义：

> `\` `+` `*` `.` `?` `-` `^` `$` `|` `(` `)` `[` `]` `{` `}` `<` `>`

当你在写 regex 的时候，如果你需要搜索的字符不是本义字符，而是以上这些特殊字符时，建议都直接加上转义符号 `\` 来表示。比如，

> * 搜索 `'` 的时候，写 `\'；
> * 搜索 `#`（事实上，`#` 并不是 regex 的特殊符号，所以它之前的转义符号可有可无）。

这对初学者来说可能是最安全的策略。

> 跟过往一样，所有的细节都很重要，它们就是需要花时间逐步熟悉到牢记。

### 集合原子

集合原子还是原子。

表示集合原子，使用方括号 `[]`。`[abc]` 的意思是说，**`a` or `b` or `c`**，即，`abc` 中的任意一个字符。

比如，`beg[iau]n` 能够代表 `begin`、`began` 以及 `begun`。

In [ ]:
import re 

strs = 'begin began begun bigins begining'
pttn = r'beg[iau]n'
re.findall(pttn, strs)

在方括号中，我们可以使用两个操作符：`-`（区间） 和 `^`（非）。

> * [a-z] 表示从小写字母 a 到小写字母 z 中的任意一个字符。
> * [^abc]表示 abc 以外的其他任意字符，即，非 `[abc]`.

注意，一个集合原子中，`^` 符号只能使用一次，只能紧跟在 `[` 之后使用，否则不起作用。

### 类别原子

类别原子，指那些能够代表**一类**字符的原子。它们都得使用转义符号再加上另外一个符号表达，包括：

> * `\d` 任意数字，等价于 [0-9]。
> * `\D` 任意非数字，等价于 [^0-9]。
> * `\w` 任意本义字符，等价于 [a-zA-Z0-9_]。
> * `\W` 任意非本义字符，等价于 `[^a-zA-Z0-9_]`。
> * `\s` 任意空白，相当于 `[ \f\n\r\t\v]`（注意，方括号内第一个字符是空格符号）。
> * `\S` 任意非空白，相当于 `[^ \f\r\t\v]`（注意，紧随 `^` 之后的是一个空格符号）。
> * `.` 除 `\r\n` 之外的任意字符，相当于 `[^\r\n]`。

类别原子挺好记忆的，如果你知道各个字母是哪个词的首字母的话：

> * `d` digits
> * `w` word characters
> * `s` spaces

另外，在，空白的集合 `[ \f\n\r\t\v]` 中，`\f` 是分页符；`\n\r` 是换行符；`\t` 是制表符；`\v` 是纵向制表符（很少用到）。各种关于空白的字符也同样挺好记忆的，如果你知道各个字母是那个词的首字母的话：

> * `f` flip
> * `n` new line
> * `r` return
> * `t` tab
> * `v` vertical tab

In [ ]:
import re
strs = '<dl>(843) 542-4256 </dl> <dl>(431) 270-9664 </dl>'
pttn = r'\d\d\d-'
re.findall(pttn, strs)

### 边界原子

可以用边界原子指定边界，也可以称为**定位操作符**。

> * `^` 匹配被搜索字符串的开始位置。
> * `$` 匹配被搜索字符串的结束位置。
> * `\b` 匹配单词的边界；`er\b`，能匹配 `wonder` 中的 `er`，却不能匹配 `error` 中的 `er`。
> * `\B` 匹配非单词的边界：`er\B`，能匹配 `error` 中的 `er`，却不能匹配 `wonder` 中的 `er`。

In [ ]:
import re

strs = 'never ever verb however everest error wonder coder'
pttn = r'er\b'
re.findall(pttn, strs)
pttn = r'er\B'
re.findall(pttn, strs)

### 组合原子

我们可以用圆括号 `()` 将多个单字组合成一个原子——这么做的结果是，`()` 内的字符串将被当作一整个原子，可以被随后我们要讲解的**数量**操作符操作。

另外，`()` 这个操作符，有两个作用：

> * 组合（grouping），就是我们刚刚讲到的作用；
> * 捕获（capturing）。

注意区别：`er`、`[er]`、`(er)` 各部相同。

> * `er` 是两个原子，`e` 和紧随其后的 `r`。
> * `[er]` 是一个原子，或者 `e` 或者 `r`。
> * `(er)` 是一个原子，`er`。

讲到数量操作符的时候，会再次强调这点。

## 数量操作符

`+` `?` `*` `{n, m}`

它们是用来限定**位于它们之前的原子允许出现的次数**；不加限定词则代表出现一次且仅出现一次。

> * `+` 代表前面的原子必须至少出现一次，即，**出现次数 $\ge$ 1** 。
>     > * 例如，`go+gle` 可以匹配 google，gooogle， goooogle 等。
> * `?` 代表前面的原子最多只能出现一次，即，**0 $\le$ 出现次数 $\le$ 1**。
>     > * 例如，`colou?red` 可以匹配 colored 或者 coloured。
> * `*` 代表前面的原子可以不出现，也可以出现一次或者多次，即，**出现次数 $\ge$ 0**.
>     > * 例如，`520*` 可以匹配 52， 520， 5200000， 5200000000000 等。
> * `{n}` 之前的原子出现确定的 n 次。
> * `{n,}` 之前的原子出现至少 n 次。
> * `{n, m}` 之前的原子出现至少 n 次，至多 m 次。
>     > * 例如，go{2,5}gle，能匹配 google gooogle， goooogle， gooooogle。但不能匹配 gogle 和 goooooogle。

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import re
with open('regex-target-text-sample.txt', 'r') as f:
    strs = f.read()
    
pttn = r'go+gle'
re.findall(pttn, strs)

pttn = r'go{2,5}gle'
re.findall(pttn, strs)

pttn = r'colou?red'
re.findall(pttn, strs)

pttn = r'520*'
re.findall(pttn, strs)

数量操作符是对它之前的原子进行操作的，换言之，**数量操作符的操作元是操作符之前的原子**。

上一节提到，注意区别 `er` `[er]` `(er)` 各不相同：

> * `er` 是两个原子，`e` 和之后的 `r`。 
> * `[er]` 是一个原子，或者 `e` 或者 `r`。
> * `(er)` 是一个原子，`er`。

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import re

strs = 'error wonderer severeness'

pttn = r'er'
re.findall(pttn, strs)

pttn = r'[er]'
re.findall(pttn, strs)

pttn = r'(er)'
re.findall(pttn, strs)

在以上的例子中，看不出 `er` 和 `(er)` 的区别。但是，加上数量操作符就不一样了——因为数量操作符只对他之前的那一个原子进行操作。

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import re

strs = 'error wonderer severness'

pttn = r'er+'
re.findall(pttn, strs)

pttn = r'[er]+'
re.findall(pttn, strs)

pttn = r'(er)+'
re.findall(pttn, strs)

## 或操作符 `|`
或操作符是所有操作符中优先级最低的，数量操作符的优先级都比它高。所以，在 `|` 前后的原子被数量操作符（如果有的话）操作之后才交给 `|` 操作。

于是，`begin|began|begun` 能够匹配 begin 或 began 或 begun。

In [ ]:
import re

strs = 'begin began begun begins beginn'
pttn = r'begin|begun|began'
re.findall(pttn, strs)

在集合原子中（即，`[]` 内的原子）各个原子之间的关系，只有**或**——相当于方括号中的每个原子之间都有一个被省略的 `|`。

**注意**：方括号中的 `|` 不被当作特殊符号，而是被当作 `|` 这个符号本身处理。在方括号中的圆括号 `()` 也被当作符号本身处理，而无分组的含义。 

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import re

strs = 'achroiocythaemia achroiocythemia a|e'
pttn = r'[a|ae]'
re.findall(pttn, strs)

pttn = r'[a|e]'
re.findall(pttn, strs)

pttn = r'[ae]'
re.findall(pttn, strs)

pttn = r'[(ae)]'
re.findall(pttn, strs)

pttn = r'[a|ae|(ae)]'
re.findall(pttn, strs)

['a', 'a', 'e', 'a', 'a', 'e', 'a', 'a', '|', 'e']

['a', 'a', 'e', 'a', 'a', 'e', 'a', 'a', '|', 'e']

['a', 'a', 'e', 'a', 'a', 'e', 'a', 'a', 'e']

['a', 'a', 'e', 'a', 'a', 'e', 'a', 'a', 'e']

['a', 'a', 'e', 'a', 'a', 'e', 'a', 'a', '|', 'e']

## 匹配并捕获

捕获（capture）使用的是圆括号 `()`。使用圆括号得到的匹配的值被暂存成一个**带有索引的列表**，第一个是 `$1`，第二个是 `$2`……以此类推。随后，我们可以在替换的过程中使用 `$1` `$2` 所保留的值。

注意：在 python 语言中调用 `re` 模块之后，在 `re.sub()` 中调用被匹配的值，用的索引方法是 `\1` `\2`…… 以此类推。

In [5]:
import re

strs = 'The white dog wears a black hat.'
pttn = r'The (white|black) dog wears a (white|black) hat.'
re.findall(pttn, strs)

repl = r'The \2 dog wears a \1 hat.'
re.sub(pttn, repl, strs)

repl = r'The \1 dog wears a \1 hat.'
re.sub(pttn, repl, strs)

[('white', 'black')]

'The black dog wears a white hat.'

'The white dog wears a white hat.'

## 非捕获匹配

有时，你并不想捕获圆括号中的内容，在那个地方你使用括号的目的只是分组，而非捕获，那么，就在圆括号内最开头加上 `?:`—— `(?:...)`：

In [11]:
import re

strs = 'The white dog wears a black hat.'
pttn = r'The (?:white|black) dog wears a (white|black) hat.'
re.findall(pttn, strs)  # 只捕获了一处，也就是说只有一个值将来可以被引用
re.match(pttn, strs)

repl = r'The \1 dog wears a \1 hat.'
re.sub(pttn, repl, strs)

['black']

<re.Match object; span=(0, 32), match='The white dog wears a black hat.'>

'The black dog wears a black hat.'

需要注意的是，虽然非匹配捕获不将匹配值暂存以便随后替换时调用，但，

> 匹配处依然是将来可被替换的位置。

在 python 代码中使用正则表达式，匹配和捕获以及随后的替换，有更灵活的方式，因为可以对那些值直接编程。`re.sub()` 中，`repl` 参数甚至可以接收另外一个函数作为参数——以后你肯定会自行认真阅读以下页面中的所有内容：

> https://docs.python.org/3/library/re.html

非捕获匹配，还有若干个操作符：

`(?=pattern)`
> 正向肯定预查（look ahead positive assert）。在任何匹配规则的字符串开始处匹配查找字符串。这是一个**非获取匹配**，即，该匹配不需要获取供以后使用。例如，`Windows(?=95|98|NT|2000%60)` 能匹配 `Windows2000` 中的 `Windows`，但不能匹配 `Windows3.1` 中的 `Windows`。预查不消耗字符，即，在一个匹配发生后，在最后一次匹配之后立即开始下一次匹配的搜索，而不是从包含预查的字符之后开始。

> 对最后一句的理解——一次匹配完成之后（问号后面的内容），下一次匹配（括号之前的内容）要从头开始搜素。

python 给的例子：

> Matches if `...` matches next, but doesn’t consume any of the string. This is called a **lookahead assertion**. For example, `Isaac (?=Asimov)` will match `'Isaac '` only if it’s followed by `'Asimov'`.

In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import re

str1 = 'Windows95. Windows2000. Windows98. Windows10. Windows3.1'
pttn = r'Windows(?=95|98|NT|2000)'
re.findall(pttn, str1)
re.sub(pttn, 'this is the matched', str1)

str2 = 'Isaac Asimov and Isaac Newton'
pttn = r'Isaac (?=Asimov)'
re.findall(pttn, str2)

['Windows', 'Windows', 'Windows']

'this is the matched95. this is the matched2000. this is the matched98. Windows10. Windows3.1'

['Isaac ']

`(?!pattern)`

> 正向否定预查（negative assert），在任何不匹配规则的字符串开始处匹配查找字符串。这是一个非获取匹配。例如，`Windows(?!95|98|NT|2000)` 能匹配 `Windows3.1` 中的 `Windows`，不能匹配 `Windows2000` 中的 `Windows`。预查不消耗字符，也就是说，在一个匹配发生之后，在最后一次匹配之后立即开始下一次匹配的搜素，而不是从包含预查的字符之后开始。

In [19]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import re

str1 = 'Windows95. Windows2000. Windows98. Windows10. Windows3.1'
pttn = r'Windows(?!95|98|NT|2000)'
re.findall(pttn, str1)
re.sub(pttn, 'this is the matched', str1)

str2 = 'Isaac Asimov and Isaac Newton'
pttn = r'Isaac (?!Asimov)'
re.findall(pttn, str2)

['Windows', 'Windows']

'Windows95. Windows2000. Windows98. this is the matched10. this is the matched3.1'

['Isaac ']

`(?<=pattern)` 

> 反向（look behind）肯定预查。与正向肯定预查类似，只是方向相反。例如，`(?<=95|98|NT|2000)Windows` 能匹配 `2000Windows` 中的 `Windows`，但不能匹配 `3.1Windows` 中的 `Windows`。

In [40]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import re

str1 = '2000Windows 95Windows 98Windows'
pttn = r'(?<=95|9)Windows'
re.findall(pttn, str1)

error: look-behind requires fixed-width pattern

`(?<!pattern)`

> 反向否定预查。与正向否定预查类似，只算方向相反。例如 `(?<!95|98|NT|2000)Windows` 能匹配 `3.1WIndows` 中的 `Windows`，但不能匹配 `2000Windows` 中的 `Windows`。

In [42]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import re

str1 = '2000Windows 95Windows 98Windows'
pttn = r'(?<!95|20)Windows'
re.findall(pttn, str1)
re.sub(pttn, 'this is the replaced place', str1)

['Windows', 'Windows']

'2000this is the replaced place 95Windows 98this is the replaced place'

## 控制标记

有几个全局控制标记（Flag）需要了解，其中最常默认指定的有 `G` 和 `M`：

`A / ASCII`，